## Multi-input Model

In [ ]:
# build multi-input model (to predict 1-word answer from relevant background text and a user question)
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

text_vocab_size = 10_000
question_vocab_size = 10_000
answer_vocab_size = 500

# input 1
text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocab_size, 64)(text_input) # the embedding arguments are backwards in the text
encoded_text = layers.LSTM(32)(embedded_text)

# input 2
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocab_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# inputs are merged via concatination into dense layer
concatinated = layers.concatenate([encoded_text, encoded_question], axis=-1)
answer = layers.Dense(answer_vocab_size, activation='softmax')(concatinated)
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
# train model on random data
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocab_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocab_size, size=(num_samples, max_length))

answers = np.zeros((num_samples, answer_vocab_size))
for answer in answers:
  answer[np.random.randint(answer_vocab_size)] = 1

# mapping data to multiple inputs:
# 1. can use a list of np arrays in the appropriate order
# OR
# 2. can use a dictionary mapping an array to each input

# 1
#model.fit([text, question], answers, epochs=10, batch_size=128)

# 2
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

## Multi-output Model

In [62]:
# build multi-input model (to predict user attributes from social media posts)
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model

vocab_size = 50_000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocab_size, 256)(posts_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# multiple outputs
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

# combine multi-output loss functions into one loss function
# default is to sum each loss, or a linear combination with weights
# weights help when different losses use different scales
# (could use lists or dictionary)
model.compile(optimizer='rmsprop',
              loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25, 'income': 1., 'gender': 10.})

In [ ]:
# train model on random data
import numpy as np

# (these must be tuned for model to work.
# e.g. if post size is too small, convolutions are too large)
num_samples = 100
post_size = 1000

posts = np.random.randint(1, vocab_size, size=(num_samples, post_size))
age_targets = np.random.randint(10, 100, size=num_samples)
income_targets = np.random.randint(0, num_income_groups, size=num_samples)
income_targets = np.eye(num_income_groups)[income_targets]
gender_targets = np.random.randint(0, 2, size=num_samples)

model.fit(posts,
          {'age': age_targets,'income': income_targets, 'gender': gender_targets},
          epochs=10, batch_size=64)